# Fine-Tune BERT LLM for Sentiment Analysis with Kubeflow PyTorchJob

This Notebook will fine-tune Bidirectional Encoder Representations from Transformers (BERT) model with Yelp dataset to analyze text sentiment using distributed training with [Kubeflow PyTorchJob](https://www.kubeflow.org/docs/components/training/overview/).

Pretrained BERT model: https://huggingface.co/google-bert/bert-base-cased

Yelp review full dataset: https://huggingface.co/datasets/yelp_review_full

This Notebook requires:

- At least **3 GPU** on your Kubernetes cluster to fine-tune BERT model on 3 workers.
- AWS S3 bucket to export fine-tuned model.

This example is based on [the HuggingFace fine-tuning tutorial](https://huggingface.co/docs/transformers/en/training).

## Install required packages

We need to install HuggingFace packages to run this Notebook.

In [ ]:
!pip install transformers datasets boto3

!pip install git+https://github.com/kubeflow/training-operator.git#subdirectory=sdk/python


## Get samples from Yelp reviews dataset

The Yelp reviews full star dataset is constructed by randomly taking 130,000 training samples and 10,000 testing samples for each review star from 1 to 5.

In total there are 650,000 training samples and 50,000 testing samples.

We are going to use this dataset to fine-tune BERT model.

In [14]:
from pprint import pprint

from datasets import load_dataset

# Test only 100 samples in the Notebook.
dataset = load_dataset("yelp_review_full", split="train[:100]")

# Print some test data.
pprint(dataset[5])
pprint(dataset[30])

{'label': 4,
 'text': "Top notch doctor in a top notch practice. Can't say I am surprised "
         'when I was referred to him by another doctor who I think is '
         'wonderful and because he went to one of the best medical schools in '
         'the country. \\nIt is really easy to get an appointment. There is '
         'minimal wait to be seen and his bedside manner is great.'}
{'label': 1,
 'text': 'Average run of the mill store.  Associates are young teens and they '
         "really don't know where anything is.  Luckily I am able to get "
         'around to find everything.  Found my puppy treats and moved on.'}


## Create script to fine-tune BERT model

We need to wrap our fine-tuning script in a function to create Kubeflow PyTorchJob.

In [4]:
def train_func(parameters):
    import os

    import boto3
    import evaluate
    import numpy as np
    from datasets import load_dataset
    from datasets.distributed import split_dataset_by_node
    from transformers import (
        AutoModelForSequenceClassification,
        AutoTokenizer,
        Trainer,
        TrainingArguments,
    )

    # [1] Download BERT model, tokenizer, and Yelp dataset.
    print("-" * 40)
    print("Download BERT Model")
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-cased",
        num_labels=5,
    )
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

    print("-" * 40)
    print("Download Yelp Review Dataset")

    # Use only 4000 data samples to reduce tokenization and training time.
    # Training samples - 3600, test samples - 400
    # Remove split to take all samples: dataset = load_dataset("yelp_review_full")
    dataset = load_dataset("yelp_review_full", split="train[:4000]")
    dataset = dataset.train_test_split(test_size=0.1, stratify_by_column="label")

    # [2] Preprocess dataset.
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    # Map Yelp review dataset to BERT tokenizer.
    print("-" * 40)
    print("Map Yelp review dataset to BERT Tokenizer")
    tokenized_ds = dataset.map(tokenize_function, batched=True)

    # Distribute train and test datasets between PyTorch workers.
    # Every worker will process chunk of training data.
    # RANK and WORLD_SIZE will be set by Kubeflow Training Operator.
    RANK = int(os.environ["RANK"])
    WORLD_SIZE = int(os.environ["WORLD_SIZE"])
    distributed_ds_train = split_dataset_by_node(
        tokenized_ds["train"],
        rank=RANK,
        world_size=WORLD_SIZE,
    )
    distributed_ds_test = split_dataset_by_node(
        tokenized_ds["test"],
        rank=RANK,
        world_size=WORLD_SIZE,
    )

    # Evaluate accuracy.
    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # [3] Define Training args.
    training_args = TrainingArguments(
        output_dir="test_trainer",
        evaluation_strategy="epoch",
        disable_tqdm=True,
        log_level="info",
    )

    # [4] Define Trainer.
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=distributed_ds_train,
        eval_dataset=distributed_ds_test,
        compute_metrics=compute_metrics,
    )

    # [5] Fine-tune model.
    print("-" * 40)
    print(f"Start Distributed Training. RANK: {RANK} WORLD_SIZE: {WORLD_SIZE}")

    trainer.train()

    print("-" * 40)
    print("Training is complete")

    # [6] Export trained model to S3 from the worker with RANK = 0.
    if RANK == 0:
        trainer.save_model("./bert")
        s3 = boto3.resource("s3")
        bucket = s3.Bucket(parameters["BUCKET"])
        bucket.upload_file("bert/config.json", "bert/config.json")
        bucket.upload_file("bert/model.safetensors", "bert/model.safetensors")

    print("-" * 40)
    print("Model is exported to S3")

## Create Kubeflow PyTorchJob to fine-tune BERT on GPUs

Use `TrainingClient()` to create PyTorchJob which will fine-tune BERT on **3 workers** using **1 GPU** for each worker.

Your Kubernetes cluster should have sufficient **GPU** resources available.

In [5]:
import uuid

# Make random name for PyTorchJob
job_name = "fine-tune-bert-" + str(uuid.uuid4())[:5]

# Replace `BUCKET_NAME` with your AWS S3 bucket.
bucket = "BUCKET_NAME"

In [6]:
from kubeflow.training import TrainingClient

# Create PyTorchJob
TrainingClient().create_job(
    name=job_name,
    train_func=train_func,
    parameters={"BUCKET": bucket},
    num_workers=3,  # Number of PyTorch workers to use.
    resources_per_worker={
        "cpu": "4",
        "memory": "10G",
        "gpu": "1",
    },
    packages_to_install=[
        "boto3",
        "transformers",
        "datasets",
        "evaluate",
        "accelerate",
        "scikit-learn",
    ],  # PIP packages will be installed during PyTorchJob runtime.
)

### Check the PyTorchJob conditions

Use `TrainingClient()` APIs to get information about created PyTorchJob.

In [7]:
print("PyTorchJob Conditions")
print(TrainingClient().get_job_conditions(job_name))
print("-" * 40)

# Wait until PyTorchJob has Running condition.
job = TrainingClient().wait_for_job_conditions(
    job_name,
    expected_conditions={"Running"},
)
print("PyTorchJob is running")

PyTorchJob Conditions
[{'last_transition_time': datetime.datetime(2024, 3, 10, 0, 37, 54, tzinfo=tzlocal()),
 'last_update_time': datetime.datetime(2024, 3, 10, 0, 37, 54, tzinfo=tzlocal()),
 'message': 'PyTorchJob fine-tune-bert-1a883 is created.',
 'reason': 'PyTorchJobCreated',
 'status': 'True',
 'type': 'Created'}, {'last_transition_time': datetime.datetime(2024, 3, 10, 0, 37, 56, tzinfo=tzlocal()),
 'last_update_time': datetime.datetime(2024, 3, 10, 0, 37, 56, tzinfo=tzlocal()),
 'message': 'PyTorchJob fine-tune-bert-1a883 is running.',
 'reason': 'PyTorchJobRunning',
 'status': 'True',
 'type': 'Running'}]
----------------------------------------
PyTorchJob is running


### Get the PyTorchJob pod names

Since we set 3 workers, PyTorchJob will create 1 master pod and 2 worker pods to execute distributed training.

In [8]:
TrainingClient().get_job_pod_names(job_name)

['fine-tune-bert-1a883-master-0',
 'fine-tune-bert-1a883-worker-0',
 'fine-tune-bert-1a883-worker-1']

### Get the PyTorchJob training logs

Every worker processes 1200 training samples on each epoch since we distribute 3600 training samples across 3 workers.

In [9]:
logs, _ = TrainingClient().get_job_logs(job_name, follow=True)


[Pod fine-tune-bert-1a883-master-0]: WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
[Pod fine-tune-bert-1a883-master-0]: ----------------------------------------
[Pod fine-tune-bert-1a883-master-0]: Download BERT Model
[Pod fine-tune-bert-1a883-master-0]: Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
[Pod fine-tune-bert-1a883-master-0]: You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[Pod fine-tune-bert-1a883-master-0]: ----------------------------------------
[Pod fine-tune-bert-1a883-master-0]: Download Yelp Review Dataset
Generating test split: 100%|██████████| 50000/50000 [00:00<00:00, 363106.11 examples/s]
[Pod fine

### Download the fine-tuned model

We can download our fine-tuned BERT model from S3 to evaluate it.

In [10]:
import boto3

s3 = boto3.resource("s3")
bucket = s3.Bucket(bucket)

# config.json is the model metadata.
# model.safetensors is the model weights & biases.
if not os.path.exists("bert"):
    os.makedirs("bert")
bucket.download_file("bert/config.json", "bert/config.json")
bucket.download_file("bert/model.safetensors", "bert/model.safetensors")

### Test the fine-tuned BERT model

We are going to use HuggingFace pipeline to test our model.

We will ask for sentiment analysis task for our fine-tuned LLM.

In [12]:
from transformers import AutoTokenizer, pipeline

# During fine-tuning BERT tokenizer is not changed.
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Use pipeline with sentiment-analysis task to evaluate our model.
nlp = pipeline("sentiment-analysis", model="./bert", tokenizer=tokenizer)

good_review = "This is one of the best restaurants I've ever been to."
bad_review = "I am upset by using this service. It is very expensive and quality is bad."

print(good_review)
res = nlp(good_review)

print("Star: ", res[0]["label"][6])
print("Score: ", res[0]["score"])
print("---------------------------\n\n")


print(bad_review)
res = nlp(bad_review)

print("Star: ", res[0]["label"][6])
print("Score: ", res[0]["score"])
print("---------------------------")

This is one of the best restaurants I've ever been to.
Star:  4
Score:  0.8029219508171082
---------------------------


I am upset by using this service. It is very expensive and quality is bad.
Star:  1
Score:  0.5185158848762512
---------------------------


## Delete the PyTorchJob

When PyTorchJob is finished, you can delete the resource.

In [13]:
TrainingClient().delete_job(name=job_name)